In [52]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sizlingdhairya1/leaf-disease")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/leaf-disease


In [53]:
! mkdir kaggle
! cp -a $path/. kaggle/

mkdir: cannot create directory ‘kaggle’: File exists


In [41]:
BASE_DIR = 'kaggle/Datasets'

In [42]:
! ls $BASE_DIR

Bacteria  Fungi  Nematodes  Normal  Virus


## Create more images by rotating and reflecting

In [50]:
! ls $BASE_DIR/Bacteria

'01a66316-0e98-4d3b-a56f-d78752cd043f___FREC_Scab 3003.JPG'
'01f3deaa-6143-4b6c-9c22-620a46d8be04___FREC_Scab 3112.JPG'
'03eccb1a-0368-4ac7-9f48-7546037b775a___FREC_Scab 3334.JPG'
'0b170906-9436-4c0d-84c1-c396ad9d909b___FREC_Scab 3101.JPG'
'0b4a52e3-e15e-4117-b2e8-7cdb5dca3ce9___FREC_Scab 3137.JPG'
'0c620ec5-11cf-4120-94ab-1311e99df147___FREC_Scab 3131.JPG'
'0cbfa4fa-63d8-43ce-9385-ff140e524b69___FREC_Scab 3164.JPG'
'0d3c0790-7833-470b-ac6e-94d0a3bf3e7c___FREC_Scab 2959.JPG'
'0d8d5b80-962d-4381-8d3b-9eca3f2f1bb0___FREC_Scab 3449.JPG'
'0db71c1d-93d7-4481-b0d0-b73f995131a9___FREC_Scab 2976.JPG'
'0e90fe4a-e8b6-4186-9429-a9fea180af9a___FREC_Scab 3391.JPG'
'0ea78733-9404-4536-8793-a108c66269b3___FREC_Scab 3145.JPG'
'1a21aabb-6f74-4644-8d9e-a517568b7e9c___FREC_Scab 3095.JPG'
'1a304331-98b5-473f-bbb1-c33b8441052a___FREC_Scab 3296.JPG'
'1a4047d7-23d6-4bba-ba10-b6e7005ea01b___FREC_Scab 3193.JPG'
'1a41bab0-45e0-4dda-a798-9bf4a998f1b6___FREC_Scab 3450.JPG'
'1b1004c8-99e9-4c85-8fe5-b1c11d558cf8___

In [44]:
from os import listdir
from PIL import Image, ImageOps

# OVERWRITE!!!
def create_img(image_path, output_path_rotated, output_path_mirrored):
    try:
        img = Image.open(image_path)

        mirrored_img = ImageOps.mirror(img)
        mirrored_img.save(output_path_mirrored)

        rotated_img = img.rotate(180, expand=True)
        rotated_img.save(output_path_rotated)

    except FileNotFoundError:
        print(f'Error: Image file not found at {image_path}')
    except Exception as e:
        print(f'An error occurred: {e}')

def new_img_name(old_name, prefix='changed'):
    return prefix + '__' + old_name


for img in listdir(f'{BASE_DIR}/Bacteria'):
    create_img(f'{BASE_DIR}/Bacteria/'+img,
               f'{BASE_DIR}/Bacteria/'+new_img_name(img, 'rotated'),
               f'{BASE_DIR}/Bacteria/'+new_img_name(img, 'mirrored'))



## Add all paths and classes to list

In [46]:
import os
from collections import defaultdict

keys = os.listdir(BASE_DIR)

data = defaultdict(list)
for cls in keys:
    img_path = os.listdir(BASE_DIR+'/'+cls)
    data['img_path'].extend(img_path)
    data['class'].extend([cls]*len(img_path))


In [47]:
import pandas as pd
import numpy as np

In [48]:
df = pd.DataFrame(data=data)
df.head()

,img_path,class
0,2e3c83d4-fbbc-4250-a3e6-96bff8d8164a___RS_GLSp...,Virus
1,3dc975e5-3570-4dbd-b163-a6f2947fc35f___RS_GLSp...,Virus
2,1effc5b5-4f56-4374-92c8-4acb0b506527___RS_GLSp...,Virus
3,2ca70127-9310-4312-8769-d9d6b43526a1___RS_GLSp...,Virus
4,5cbbb4b8-88cc-4d5f-907a-505a2a831b8a___RS_GLSp...,Virus


In [49]:
df['class'].value_counts()

,count
class,
Bacteria,100
Virus,50
Fungi,50
Nematodes,49
Normal,40
